In [6]:
import argparse
parser = argparse.ArgumentParser()

#=================TRAINING PARAMETERS=================
parser.add_argument("--num_frames", dest='num_frames', type=int, required=False, default=16, help='Number of frames')
parser.add_argument("--sample_every_n_frames", dest='sample_every_n_frames', type=int, required=False, default=2, help='Sample every n frames')
parser.add_argument("--num_workers", dest='num_workers', type=int, required=False, default=10, help='Number of workers')
parser.add_argument("--batch_size", dest='batch_size', type=int, required=False, default=32, help='Batch size')
parser.add_argument("--v_batch_size", dest='v_batch_size', type=int, required=False, default=8, help='Validation batch size')
parser.add_argument("--learning_rate", dest='learning_rate', type=float, required=False, default=1e-4, help='Learning rate')
parser.add_argument("--num_epochs", dest='num_epochs', type=int, required=False, default=300, help='Number of epochs')
parser.add_argument("--val_freq", dest='val_freq', type=int, required=False, default=10, help='Validation frequency')
#=================DATA PARAMETERS=================
parser.add_argument("--reso_h", dest='reso_h', type=int, required=False, default=128, help='Resolution height')
parser.add_argument("--reso_w", dest='reso_w', type=int, required=False, default=128, help='Resolution width')
parser.add_argument("--triple", dest='triple', type=int, required=False, default=1, help='Use triple sampling') #用于控制是否使用triplet loss
#=================VQGAN PARAMETERS=================
parser.add_argument('--embedding_dim_dynamic', type=int, default=256)
parser.add_argument('--embedding_dim_static', type=int, default=256)
parser.add_argument('--n_codes_dynamic', type=int, default=16384)
parser.add_argument('--n_codes_static', type=int, default=2048)
parser.add_argument('--n_hiddens', type=int, default=32)
parser.add_argument('--downsample', nargs='+', type=int, default=(4, 8, 8))
parser.add_argument('--image_channels', type=int, default=3)
parser.add_argument('--disc_channels', type=int, default=64)
parser.add_argument('--disc_layers', type=int, default=3)
parser.add_argument('--discriminator_iter_start', type=int, default=10)
parser.add_argument('--triplet_iter_start', type=int, default=20) # 用于控制什么时候开始使用triplet loss来分离动态和静态信息
parser.add_argument('--disc_loss_type', type=str, default='hinge', choices=['hinge', 'vanilla'])
parser.add_argument('--image_gan_weight', type=float, default=1.0)
parser.add_argument('--video_gan_weight', type=float, default=1.0)
parser.add_argument('--l1_weight', type=float, default=4.0)
parser.add_argument('--gan_feat_weight', type=float, default=4.0)
parser.add_argument('--perceptual_weight', type=float, default=4.0)
parser.add_argument('--restart_thres', type=float, default=1.0)
parser.add_argument('--no_random_restart', action='store_true')
parser.add_argument('--norm_type', type=str, default='batch', choices=['batch', 'group'])
parser.add_argument('--padding_type', type=str, default='replicate', choices=['replicate', 'constant', 'reflect', 'circular'])

opts = parser.parse_args([])

In [ ]:
import torch
from models.vqgan import VQGAN
# 假设模型和数据已经准备好
model = VQGAN(opts)  # 你的VQGAN模型
model.train()  # 确保在训练模式

In [ ]:





# 2. 保存更新前的参数副本
params_before = {}
for name, param in model.named_parameters():
    params_before[name] = param.clone().detach()

# 3. 执行一次前向传播和反向传播
optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=0.001)

# 假设输入数据形状为 [batch_size, channels, time, height, width]
x = torch.randn(2, 3, 16, 64, 64).cuda()  # 根据你的实际输入调整

# 前向传播
recon_loss, commitment_loss_static, commitment_loss_dynamic, aeloss, perceptual_loss, gan_feat_loss = model(x, optimizer_idx=0)

# 计算总损失
total_loss = recon_loss + commitment_loss_dynamic + commitment_loss_static + aeloss + perceptual_loss + gan_feat_loss

# 反向传播和优化
optimizer.zero_grad()
total_loss.backward()
optimizer.step()

# 4. 检查参数是否更新
print("\n参数更新检查:")
for name, param in model.named_parameters():
    # 计算参数变化
    param_change = torch.sum(torch.abs(param.data - params_before[name])).item()
    
    print(f"{name}:")
    print(f"  requires_grad: {param.requires_grad}")
    print(f"  参数变化量: {param_change}")
    print(f"  是否更新: {'是' if param_change > 0 else '否'}")
    print()

In [ ]:
import torch
# 设置随机数种子
torch.manual_seed(42)  # 使用任意整数作为种子
# 如果在分布式环境中运行，请确保所有设备的种子一致
torch.cuda.manual_seed_all(42)  # 确保在每个GPU上都设定相同的种子

# 如果使用GPU，也设置GPU上的随机数种子
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(42)
for i in range(3):
    # 使用 torch.randint 生成确定性的随机数
    random_tensor = torch.randint(0, 10, (3, 3))
    random_tensor_1 = torch.randn(2,4)
    print(random_tensor)
    print(random_tensor_1)